In [ ]:
import os
from os import path
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), '../..'))

import db_builder.db_handler as dbh

import numpy as np
import cv2
from torchvision import transforms
from tqdm import tqdm
import datetime


In [ ]:
db_params = {
    'user': 'postgres',
    'password': input('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase', #tensionTerminator
}

In [ ]:
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
filepath_with_trigger = toolcheck.get_filepath_with_trigger()
filepath_with_duoballs = toolcheck.get_filepath_with_duoballs()
toolcheck.disconnect()

In [ ]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [ ]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 1

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [ ]:
def get_time():
    return datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

In [ ]:
def save_path(label: str, time):
    abs_path = os.getcwd()
    two_up =  path.abspath(path.join(abs_path ,"../.."))
    return path.join(two_up, 'data', f'{os.path.basename(abs_path)}', f'{time}', label)

In [ ]:
current_time = get_time()

In [ ]:
for x in tqdm(filepath_with_trigger):
    video_to_image_converter(x, save_path('trigger', current_time), False)

In [ ]:
for x in tqdm(filepath_with_duoballs):
    video_to_image_converter(x, save_path('duoballs', current_time), False)